In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [53]:
fips_check = pd.read_csv("data/state_data/geo/geo_pa.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #38537

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,Pittsburgh,PA,...,Unanswered,NaN,10114794.52,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512
1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,HARRISBURG,PA,...,Unanswered,NaN,10111232.88,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894
2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,PITTSBURGH,PA,...,Unanswered,NaN,NaN,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727
3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,King of Prussia,PA,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199
4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,HOMESTEAD,PA,...,Unanswered,NaN,10120277.78,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38532,774584,774584,9738208500,3/12/21,358,PPS,OESTERLING'S SANDBLASTING & PAINTING INC.,686 Glenwood Way,Butler,PA,...,Non-Veteran,NaN,150916.44,10/26/21,21.0,Mining,PITTSBURGH DISTRICT OFFICE,"686 Glenwood Way, Butler, PA",40.904294,-79.887279
38533,774585,774585,9754548410,2/17/21,358,PPP,PHARMACY HEALTHCARE SOLUTIONS LLC,968 Perry Hwy 968 Perry Hwy,Pittsburgh,PA,...,Unanswered,NaN,150954.17,10/7/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949
38534,774586,774586,9876218305,1/31/21,303,PPS,NATIONAL ASSOCIATION OF WATCH AND CLOCK COLLEC...,514 Poplar St,Columbia,PA,...,Unanswered,Y,150932.88,9/17/21,71.0,"Arts, Entertainment, and Recreation",PHILADELPHIA DISTRICT OFFICE,"514 Poplar St, Columbia, PA",40.037187,-76.502122
38535,774587,774587,9910327007,4/9/20,303,PPP,"MAA AMBE FOODS, LLC",900 Lehigh Valley Mall Dr.,WHITEHALL,PA,...,Non-Veteran,NaN,151360.27,4/2/21,99.0,0,PHILADELPHIA DISTRICT OFFICE,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #8

,LoanNumber,full_add,Lat,Long
7106,5889008400,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",NaN,NaN
13461,1563267305,"417 N. 8th. Street, Philadelphia, PA",NaN,NaN
25410,6128077109,"99 CENTER ST, EAST PITTSBURGH, PA",NaN,NaN
27367,7589518506,"417 N 8th St Fl 4 417 N 8th St Fl 4, Philadelp...",NaN,NaN
30888,9549038607,"44 Hersha Dr, Harrisburg, PA",NaN,NaN
31757,5995597007,"PO BOX 128, BETHEL, PA",NaN,NaN
35168,3618607210,"3375 Lynnwood Drive, Altoona, PA",NaN,NaN
35900,3448828307,"706, BLUE BELL, PA",NaN,NaN


In [6]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #

,LoanNumber,full_add,Lat,Long
0,1780617201,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512
1,1828167306,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894
2,2394397105,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727
3,4165028809,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199
4,4396367001,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/PA_fips_scraped.csv") 
fips_check_scraped.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,...,NaN,10114794.52,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,1,1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,...,NaN,10111232.88,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,2,2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,...,NaN,NaN,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,3,3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,4,4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,...,NaN,10120277.78,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna[:1030] #38529

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1780617201,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,1828167306,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,2394397105,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,4165028809,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,4396367001,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14
...,...,...,...,...,...
1025,4550497102,"500 E SWEDESFORD RD, WAYNE, PA",40.076582,-75.420315,4.202930e+14
1026,8609117108,"1425 West Main Street, Mount Joy, PA",40.120889,-76.539520,4.207101e+14
1027,9490397107,"104 Independence Way Coatesville, Coatesville, PA",39.984163,-75.881988,4.202931e+14
1028,1974357204,"275 SITHAN AVE, BRYN MAWR, PA",40.028352,-75.349872,4.204541e+14


In [6]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #2828

,LoanNumber,full_add,Lat,Long,FIPS_z
943,6710297002,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
4243,7276257209,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5637,3628157101,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6995,7706207108,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7703,3639927102,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
...,...,...,...,...,...
38532,9738208500,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,NaN
38533,9754548410,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,NaN
38534,9876218305,"514 Poplar St, Columbia, PA",40.037187,-76.502122,NaN
38535,9910327007,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,NaN


<h4>FIPS Script

In [7]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('42')] #PA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

2828


Matching FIPS to DataFrame

In [11]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:17642] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-12-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
943,6710297002,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,[]
4243,7276257209,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,[]
5637,3628157101,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,[]
6995,7706207108,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,[]
7703,3639927102,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,[]
...,...,...,...,...,...
38532,9738208500,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,[420199108001041]
38533,9754548410,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,[420034294002006]
38534,9876218305,"514 Poplar St, Columbia, PA",40.037187,-76.502122,[420710113002019]
38535,9910327007,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,[420770057022047]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-13-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
943,6710297002,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
4243,7276257209,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5637,3628157101,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6995,7706207108,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7703,3639927102,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
...,...,...,...,...,...
38532,9738208500,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,420199108001041
38533,9754548410,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,420034294002006
38534,9876218305,"514 Poplar St, Columbia, PA",40.037187,-76.502122,420710113002019
38535,9910327007,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,420770057022047


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #29

,LoanNumber,full_add,Lat,Long,FIPS_z
943,6710297002,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
4243,7276257209,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5637,3628157101,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6995,7706207108,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7703,3639927102,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
8695,7421237102,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN
9602,3026777102,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN
10078,4466207104,"208 Flynn Ave., Suite 2A, UNIONTOWN, PA",44.455988,-73.217960,NaN
12660,3687147708,"3821 South Rd, Mukilteo, PA",47.883043,-122.285864,NaN
13304,4892717303,"570 Route 202, North Wales, PA",41.506985,-73.420491,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
943,6710297002,NaN
4243,7276257209,NaN
5637,3628157101,NaN
6995,7706207108,NaN
7703,3639927102,NaN
...,...,...
38532,9738208500,420199108001041
38533,9754548410,420034294002006
38534,9876218305,420710113002019
38535,9910327007,420770057022047


Read in base data

In [18]:
#if from geocode file
#fips_check = pd.read_csv("data/state_data/geo/geo_pa.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/PA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #38537


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,...,NaN,10114794.52,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,1,1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,...,NaN,10111232.88,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,2,2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,...,NaN,NaN,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,3,3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,4,4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,...,NaN,10120277.78,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14


Merge with MainFrame

In [19]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,...,10114794.52,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14,NaN
1,1,1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,...,10111232.88,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14,NaN
2,2,2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,...,NaN,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14,NaN
3,3,3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,...,NaN,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14,NaN
4,4,4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,...,10120277.78,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38532,38532,38532,774584,774584,9738208500,3/12/21,358,PPS,OESTERLING'S SANDBLASTING & PAINTING INC.,686 Glenwood Way,...,150916.44,10/26/21,21.0,Mining,PITTSBURGH DISTRICT OFFICE,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,NaN,420199108001041
38533,38533,38533,774585,774585,9754548410,2/17/21,358,PPP,PHARMACY HEALTHCARE SOLUTIONS LLC,968 Perry Hwy 968 Perry Hwy,...,150954.17,10/7/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,NaN,420034294002006
38534,38534,38534,774586,774586,9876218305,1/31/21,303,PPS,NATIONAL ASSOCIATION OF WATCH AND CLOCK COLLEC...,514 Poplar St,...,150932.88,9/17/21,71.0,"Arts, Entertainment, and Recreation",PHILADELPHIA DISTRICT OFFICE,"514 Poplar St, Columbia, PA",40.037187,-76.502122,NaN,420710113002019
38535,38535,38535,774587,774587,9910327007,4/9/20,303,PPP,"MAA AMBE FOODS, LLC",900 Lehigh Valley Mall Dr.,...,151360.27,4/2/21,99.0,0,PHILADELPHIA DISTRICT OFFICE,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,NaN,420770057022047


Fill FIPS Columns - if FIPS file

In [20]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,...,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14,NaN,4.2003e+14
1,1,1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,...,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14,NaN,4.2043e+14
2,2,2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,...,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14,NaN,4.2003e+14
3,3,3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,...,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14,NaN,4.20912e+14
4,4,4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,...,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14,NaN,4.20035e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38532,38532,38532,774584,774584,9738208500,3/12/21,358,PPS,OESTERLING'S SANDBLASTING & PAINTING INC.,686 Glenwood Way,...,10/26/21,21.0,Mining,PITTSBURGH DISTRICT OFFICE,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,NaN,420199108001041,420199108001041
38533,38533,38533,774585,774585,9754548410,2/17/21,358,PPP,PHARMACY HEALTHCARE SOLUTIONS LLC,968 Perry Hwy 968 Perry Hwy,...,10/7/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,NaN,420034294002006,420034294002006
38534,38534,38534,774586,774586,9876218305,1/31/21,303,PPS,NATIONAL ASSOCIATION OF WATCH AND CLOCK COLLEC...,514 Poplar St,...,9/17/21,71.0,"Arts, Entertainment, and Recreation",PHILADELPHIA DISTRICT OFFICE,"514 Poplar St, Columbia, PA",40.037187,-76.502122,NaN,420710113002019,420710113002019
38535,38535,38535,774587,774587,9910327007,4/9/20,303,PPP,"MAA AMBE FOODS, LLC",900 Lehigh Valley Mall Dr.,...,4/2/21,99.0,0,PHILADELPHIA DISTRICT OFFICE,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,NaN,420770057022047,420770057022047


Drop Extra FIPS Columns

In [21]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,736052,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,2767 Liberty Avenue,...,NaN,10114794.52,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.2003e+14
1,1,1,736053,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",3940 Locust Lane,...,NaN,10111232.88,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.2043e+14
2,2,2,736054,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",2501 Smallman Street,...,NaN,NaN,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.2003e+14
3,3,3,736055,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,2012 Renaissance Blvd,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.20912e+14
4,4,4,736056,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",285 E Waterfront Dr,...,NaN,10120277.78,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.20035e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38532,38532,38532,774584,774584,9738208500,3/12/21,358,PPS,OESTERLING'S SANDBLASTING & PAINTING INC.,686 Glenwood Way,...,NaN,150916.44,10/26/21,21.0,Mining,PITTSBURGH DISTRICT OFFICE,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,420199108001041
38533,38533,38533,774585,774585,9754548410,2/17/21,358,PPP,PHARMACY HEALTHCARE SOLUTIONS LLC,968 Perry Hwy 968 Perry Hwy,...,NaN,150954.17,10/7/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,420034294002006
38534,38534,38534,774586,774586,9876218305,1/31/21,303,PPS,NATIONAL ASSOCIATION OF WATCH AND CLOCK COLLEC...,514 Poplar St,...,Y,150932.88,9/17/21,71.0,"Arts, Entertainment, and Recreation",PHILADELPHIA DISTRICT OFFICE,"514 Poplar St, Columbia, PA",40.037187,-76.502122,420710113002019
38535,38535,38535,774587,774587,9910327007,4/9/20,303,PPP,"MAA AMBE FOODS, LLC",900 Lehigh Valley Mall Dr.,...,NaN,151360.27,4/2/21,99.0,0,PHILADELPHIA DISTRICT OFFICE,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,420770057022047


In [29]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #41 , notna #38496 / 38537

,Unnamed: 0,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
943,943,736995,6710297002,4/7/20,303,PPP,TECHNOLOGY RESOURCE GROUP LLC,3001 W Big Beaver Rd,SUMNEYTOWN,PA,...,6/14/21,54.0,"Professional, Scientific, and Technical Services",PHILADELPHIA DISTRICT OFFICE,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN,NaN,NaN
4243,4243,740295,7276257209,4/28/20,358,PPP,"NFO, INC.",528 Billy Sunday Rd,PITTSBURGH,PA,...,11/30/20,42.0,Wholesale Trade,PITTSBURGH DISTRICT OFFICE,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN,NaN,NaN
5637,5637,741689,3628157101,4/11/20,303,PPP,DENHOLZ MANAGEMENT CO.,116 Chestnut St. 102,Red Bank,PA,...,3/2/21,53.0,Real Estate Rental and Leasing,PHILADELPHIA DISTRICT OFFICE,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN,NaN,NaN
6995,6995,743047,7706207108,4/14/20,303,PPP,VIONX ENERGY CORPORATION,12a Cabot Rd,ALBURTIS,PA,...,NaN,33.0,Manufacturing,PHILADELPHIA DISTRICT OFFICE,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN,NaN,NaN
7106,7106,743158,5889008400,2/9/21,303,PPS,PELAS CORP,417 N 8th St Ste 302 417 N 8th St Ste 302,Philadelphia,PA,...,NaN,42.0,Wholesale Trade,PHILADELPHIA DISTRICT OFFICE,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",NaN,NaN,NaN,NaN,NaN
7703,7703,743755,3639927102,4/11/20,303,PPP,"PIPELINE PETROLEUM, INC.","1 Tank Farm and Shippers Rd, box 159",MACUNGIE,PA,...,6/17/21,42.0,Wholesale Trade,PHILADELPHIA DISTRICT OFFICE,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN,NaN,NaN
8695,8695,744747,7421237102,4/14/20,303,PPP,"WARREN GROUP INC, THE",2 Corporation Way Ste 250,READING,PA,...,3/31/21,51.0,Information,PHILADELPHIA DISTRICT OFFICE,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN,NaN,NaN
9602,9602,745654,3026777102,4/11/20,303,PPP,LYOPHILIZATION TECHNOLOGY INC,20 Indian Dr,Langhorne,PA,...,8/19/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN,NaN,NaN
10078,10078,746130,4466207104,4/13/20,358,PPP,"ENGINEERING VENTURES, PC","208 Flynn Ave., Suite 2A",UNIONTOWN,PA,...,7/19/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"208 Flynn Ave., Suite 2A, UNIONTOWN, PA",44.455988,-73.217960,NaN,NaN,NaN
12660,12660,748712,3687147708,5/1/20,303,PPP,SQI INCORPORATED,3821 South Rd,Mukilteo,PA,...,7/6/21,23.0,Construction,PHILADELPHIA DISTRICT OFFICE,"3821 South Rd, Mukilteo, PA",47.883043,-122.285864,NaN,NaN,NaN


In [25]:
fips_PA = fips_merge_drop

Convert to File

In [4]:
fips_PA.to_csv("data/state_data/geo/geo_fips/PA_fips_scraped.csv") 

Review & Compare

In [3]:
geo_PA_fips = pd.read_csv("data/state_data/geo/geo_fips/PA_fips_scraped.csv") 
geo_PA_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,0,0,736052,1780617201,4/15/20,358,PPP,SARGENT ELECTRIC COMPANY,...,6/11/21,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14,NaN,4.200302e+14
1,1,1,1,1,736053,1828167306,4/28/20,303,PPP,"PRIMECARE MEDICAL, INC.",...,6/11/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14,NaN,4.204302e+14
2,2,2,2,2,736054,2394397105,4/10/20,358,PPP,"RYCON CONSTRUCTION, INC.",...,NaN,23.0,Construction,PITTSBURGH DISTRICT OFFICE,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14,NaN,4.200302e+14
3,3,3,3,3,736055,4165028809,4/15/21,303,PPP,THE DEVEREUX FOUNDATION,...,NaN,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14,NaN,4.209121e+14
4,4,4,4,4,736056,4396367001,4/3/20,358,PPP,"EAT'N PARK HOSPITALITY, INC.",...,6/11/21,72.0,Accomodation and Food Services,PITTSBURGH DISTRICT OFFICE,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14,NaN,4.200348e+14


In [4]:
#fill in missing Lat, Long, FIPS
geo_PA_fips.loc[geo_PA_fips["FIPS_z"].isnull()] #2836

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
943,943,943,943,943,736995,6710297002,4/7/20,303,PPP,TECHNOLOGY RESOURCE GROUP LLC,...,6/14/21,54.0,"Professional, Scientific, and Technical Services",PHILADELPHIA DISTRICT OFFICE,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN,NaN,NaN
4243,4243,4243,4243,4243,740295,7276257209,4/28/20,358,PPP,"NFO, INC.",...,11/30/20,42.0,Wholesale Trade,PITTSBURGH DISTRICT OFFICE,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN,NaN,NaN
5637,5637,5637,5637,5637,741689,3628157101,4/11/20,303,PPP,DENHOLZ MANAGEMENT CO.,...,3/2/21,53.0,Real Estate Rental and Leasing,PHILADELPHIA DISTRICT OFFICE,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN,NaN,NaN
6995,6995,6995,6995,6995,743047,7706207108,4/14/20,303,PPP,VIONX ENERGY CORPORATION,...,NaN,33.0,Manufacturing,PHILADELPHIA DISTRICT OFFICE,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN,NaN,NaN
7106,7106,7106,7106,7106,743158,5889008400,2/9/21,303,PPS,PELAS CORP,...,NaN,42.0,Wholesale Trade,PHILADELPHIA DISTRICT OFFICE,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",NaN,NaN,NaN,NaN,NaN
7703,7703,7703,7703,7703,743755,3639927102,4/11/20,303,PPP,"PIPELINE PETROLEUM, INC.",...,6/17/21,42.0,Wholesale Trade,PHILADELPHIA DISTRICT OFFICE,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN,NaN,NaN
8695,8695,8695,8695,8695,744747,7421237102,4/14/20,303,PPP,"WARREN GROUP INC, THE",...,3/31/21,51.0,Information,PHILADELPHIA DISTRICT OFFICE,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN,NaN,NaN
9602,9602,9602,9602,9602,745654,3026777102,4/11/20,303,PPP,LYOPHILIZATION TECHNOLOGY INC,...,8/19/21,62.0,Health Care and Social Assistance,PHILADELPHIA DISTRICT OFFICE,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN,NaN,NaN
10078,10078,10078,10078,10078,746130,4466207104,4/13/20,358,PPP,"ENGINEERING VENTURES, PC",...,7/19/21,54.0,"Professional, Scientific, and Technical Services",PITTSBURGH DISTRICT OFFICE,"208 Flynn Ave., Suite 2A, UNIONTOWN, PA",44.455988,-73.217960,NaN,NaN,NaN
12660,12660,12660,12660,12660,748712,3687147708,5/1/20,303,PPP,SQI INCORPORATED,...,7/6/21,23.0,Construction,PHILADELPHIA DISTRICT OFFICE,"3821 South Rd, Mukilteo, PA",47.883043,-122.285864,NaN,NaN,NaN


In [4]:
geo_PA_fips["BusinessType"] = geo_PA_fips["BusinessType"].fillna('Not Available')
geo_PA_fips["BusinessType"].unique()
geo_PA_fips.to_csv("data/state_data/geo/geo_fips/PA_fips_scraped.csv") 
geo_PA_fips["BusinessType"].unique()

array(['Subchapter S Corporation', '501(c)3 – Non Profit', 'Corporation',
       'Partnership', 'Professional Association',
       'Limited  Liability Company(LLC)', 'Sole Proprietorship',
       'Non-Profit Organization', 'Limited Liability Partnership',
       'Trust', 'Employee Stock Ownership Plan(ESOP)',
       'Non-Profit Childcare Center', 'Cooperative', 'Not Available',
       'Independent Contractors', '501(c)6 – Non Profit Membership',
       'Joint Venture', 'Self-Employed Individuals',
       '501(c) – Non Profit except 3,4,6,', 'Single Member LLC',
       'Housing Co-op'], dtype=object)